### Loading the dataset

In [62]:
!pip uninstall jedi
from datasets import load_dataset
from pathlib import Path
import os

In [4]:
# reading the jsonl dataset
data_dir = Path("../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl")

import pandas as pd    
jsonObj = pd.read_json(path_or_buf=data_dir, lines=True)



In [5]:
jsonObj.head()["answer"][2]

'reject'

In [6]:
jsonObj.columns

Index(['text', 'meta', '_input_hash', '_task_hash', 'label', 'score',
       'priority', 'spans', '_session_id', '_view_id', 'answer'],
      dtype='object')

### Huggingface dataset loading script
can be used later to publish the dataset on huggingface hub for better outreach

### Testing the dataset loading script

In [108]:
from datasets import load_dataset


dataset = load_dataset("my_dataset_loading_script", name="condemnation")

0 examples [00:00, ? examples/s]

filepath my_dataset_loading_script/condemnation_train.csv


0 examples [00:00, ? examples/s]

filepath my_dataset_loading_script/condemnation_test.csv
Dataset condemnation_dataset downloaded and prepared to /home/geev/.cache/huggingface/datasets/condemnation_dataset/condemnation/1.1.0/4316c50c4e4ffaba14acbd793416d72132d364d43fd27c03abe2ff72eab3c9fc. Subsequent calls will reuse this data.


In [111]:
train_dataset= dataset["train"]

In [112]:
train_dataset.shape

(1344, 2)

In [113]:
train_dataset[0]

{'text': 'As multiple sexual allegations come to light with people like <OTHER TARGET 1>, NPR’s -JOHN DOE-, and Michael Fallon, just to name a few...',
 'label': 1}

In [114]:
train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [115]:
train_dataset.dataset_size

227819

In [116]:
# train_dataset.set_format(type="torch", columns=["label"])
train_dataset[:10]

OrderedDict([('text',
              ['As multiple sexual allegations come to light with people like <OTHER TARGET 1>, NPR’s -JOHN DOE-, and Michael Fallon, just to name a few...',
               'Matt Damon and -JOHN DOE- are trash',
               'Thought we all knew tht -JOHN DOE- is a narcissistic asshole??',
               '-JOHN DOE- you disgusting pig!',
               'Breaking #FoxNews Alert : Amazon executive -JOHN DOE- suspended as sexual harassment accusations surface',
               "The more stories of sexual assault I hear, the more I fear raising a child in today's world. #<OTHER TARGET 1> #<OTHER TARGET 2> #-JOHN DOE- #Trump #Oreilly",
               'In -JOHN DOE-’s worldview, REAL men drug and violently rape women like he does. As a writer, he’s done. But he might have a future in the GOP…',
               'im just reading about -JOHN DOE- and i cant believe garbage created something that changed my life and gave me so much happiness',
               "-JOHN DOE- and

In [117]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def encode(batch):
    return tokenizer(batch["text"], padding="longest", truncation=True, max_length=512, return_tensors="pt")
# train_dataset.set_format(type="torch", columns=["label"], output_all_columns=True)
train_dataset.set_transform(encode, columns=["text"], output_all_columns=True)


In [118]:
train_dataset[0]

{'input_ids': tensor([  101,  2004,  3674,  4424,  9989,  2272,  2000,  2422,  2007,  2111,
          2066,  1026,  2060,  4539,  1015,  1028,  1010, 21411,  1521,  1055,
          1011,  2198, 18629,  1011,  1010,  1998,  2745, 16443,  1010,  2074,
          2000,  2171,  1037,  2261,  1012,  1012,  1012,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'label': 1}

In [119]:
train_dataset.reset_format()
train_dataset[0]

{'text': 'As multiple sexual allegations come to light with people like <OTHER TARGET 1>, NPR’s -JOHN DOE-, and Michael Fallon, just to name a few...',
 'label': 1}

### Generating the Train-Test Split
-----------------------------------------------------------------------------------------------------------

In [82]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
data_dir = "my_dataset_loading_script"
file_name = "nour_jillian_condemnation_r2_to_r7_maj_vote.jsonl"
def create_train_test(data_dir, file_name):
    file_path = os.path.join(data_dir, file_name )
    ds_df = pd.read_json(file_path, lines=True)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    sss.get_n_splits(ds_df, ds_df["label"])
    print(sss)
#     print(ds_df)
    for train_index, test_index in sss.split(ds_df, ds_df["label"]):
        print(train_index)
        train_df= ds_df.iloc[train_index,:]
        test_df= ds_df.iloc[test_index,:]

#         test_df = ds_df[test_index]
    return train_df, test_df
train_df, test_df = create_train_test(data_dir, file_name)
        

StratifiedShuffleSplit(n_splits=1, random_state=0, test_size=0.2,
            train_size=None)
[1509  415  988 ...  864 1143   91]


In [85]:
sum(train_df.label)/len(train_df.label)

0.6264880952380952

In [86]:
sum(test_df.label)/len(test_df.label)

0.6279761904761905

In [87]:
len(train_df)

1344

In [88]:
len(test_df)

336

In [101]:
ARE_U_SURE = False
if ARE_U_SURE:
    train_df.to_csv("my_dataset_loading_script/condemnation_train.csv", index=False)
    test_df.to_csv("my_dataset_loading_script/condemnation_test.csv", index=False)
else:
    print("This will regenerate the test set! are you sure?")

------------------------------------------------------------